# Remote data access using pandas

The pandas library enables access to data displayed on websites using the `read_html()` function and access to the API endpoints of various data providers through the related `pandas-datareader` library.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas_datareader.data as web


In [ ]:
!python -m pip install --upgrade pip


In [ ]:
!pip install mplfinance


In [ ]:
%matplotlib inline
import os
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns

## Download html table with SP500 constituents

The download of the content of one or more html tables works as follows, for instance for the constituents of the S&P500 index from Wikipedia

In [ ]:
import pandas as pd
import requests

sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(sp_url)
tables = pd.read_html(response.text)

sp500_constituents = tables[0]
print(sp500_constituents.head())


In [ ]:
sp500_constituents.info()

In [ ]:
sp500_constituents.head()

## pandas-datareader for Market Data

`pandas` used to facilitate access to data providers' APIs directly, but this functionality has moved to the related pandas-datareader library. The stability of the APIs varies with provider policies, and as of June 2o18 at version 0.7, the following sources are available

See [documentation](https://pandas-datareader.readthedocs.io/en/latest/); functionality frequently changes as underlying provider APIs evolve.

### Yahoo Finance

In [ ]:
!pip install yfinance


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start = '2014-01-01'
end = '2017-05-24'

# Use yfinance to download data for Meta (formerly Facebook)
yahoo = yf.download('META', start=start, end=end)

# Show DataFrame
print(yahoo.head())



In [ ]:
import yfinance as yf
import pandas as pd

start = '2014-01-01'
end = '2017-05-24'

# Download data for META (formerly FB)
yahoo = yf.download('META', start=start, end=end)


In [ ]:
# Flatten column MultiIndex
yahoo.columns = yahoo.columns.get_level_values(0)


In [ ]:
# Only drop 'Adj Close' if it exists
if 'Adj Close' in yahoo.columns:
    yahoo = yahoo.drop('Adj Close', axis=1)


### IEX

IEX is an alternative exchange started in response to the HFT controversy and portrayed in Michael Lewis' controversial Flash Boys. It aims to slow down the speed of trading to create a more level playing field and has been growing rapidly since launch in 2016 while still small with a market share of around 2.5% in June 2018.

> **Note:** IEX now requires an [API](https://iexcloud.io/) key after registration for (free) account that you can store as environment variable and retrieve as illustrated below, or pass directly via keyword argument to `pandas_datareader`.

In [ ]:
IEX_API_KEY=os.getenv('IEX_API_KEY')

In [ ]:
import yfinance as yf
import mplfinance as mpf
import pandas as pd

# Step 1: Download data for META (Facebook)
start = '2015-02-09'
end = '2017-05-24'
yahoo = yf.download('META', start=start, end=end)

# Step 2: Ensure column names are clean
if isinstance(yahoo.columns, pd.MultiIndex):
    yahoo.columns = yahoo.columns.get_level_values(0)

# Step 3: Set index name for mplfinance
yahoo.index.name = 'Date'

# Step 4: Plot candlestick chart with volume
mpf.plot(yahoo, type='candle', volume=True, title='META Candlestick Chart')


In [ ]:
yahoo.tail()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

# Plot 'Close' prices from yahoo instead of iex
yahoo['Close'].plot(figsize=(14, 5), title='META Close Price')
sns.despine()
plt.show()


#### Book Data

In addition to historical EOD price and volume data, IEX provides real-time depth of book quotations that offer an aggregated size of orders by price and side. This service also includes last trade price and size information.

DEEP is used to receive real-time depth of book quotations direct from IEX. The depth of book quotations received via DEEP provide an aggregated size of resting displayed orders at a price and side, and do not indicate the size or number of individual orders at any price level. Non-displayed orders and non-displayed portions of reserve orders are not represented in DEEP.

DEEP also provides last trade price and size information. Trades resulting from either displayed or non-displayed orders matching on IEX will be reported. Routed executions will not be reported.

Only works on trading days.

In [ ]:
import yfinance as yf

data = yf.Ticker("AAPL")
book_data = data.history(period="1d", interval="1m")
print(book_data.tail())


In [ ]:
list(book_data.keys())


In [ ]:
# Using dummy order book data since 'book' was not defined earlier
# 'bids' and 'asks' represent price and volume levels on each side of the order book

book = {
    'bids': [[100.5, 200], [100.4, 150], [100.3, 100]],
    'asks': [[100.6, 250], [100.7, 180], [100.8, 120]]
}

# Combine bids and asks into a single DataFrame with a 'side' column
orders = pd.concat([
    pd.DataFrame(book[side], columns=['price', 'volume']).assign(side=side)
    for side in ['bids', 'asks']
])

orders.head()


In [ ]:
for key in book.keys():
    try:
        print(f'\n{key}')
        print(pd.DataFrame(book[key]))
    except:
        print(book[key])

In [ ]:
# Add dummy 'trades' data to the book
book['trades'] = [
    {'price': 100.55, 'volume': 50, 'timestamp': '2025-07-15 10:00:01'},
    {'price': 100.60, 'volume': 30, 'timestamp': '2025-07-15 10:00:02'},
    {'price': 100.58, 'volume': 20, 'timestamp': '2025-07-15 10:00:03'}
]


In [ ]:
pd.DataFrame(book['trades']).head()

### Quandl

Obtain Quandl [API Key](https://www.quandl.com/tools/api) and store in environment variable as `QUANDL_API_KEY`.

In [ ]:
pip install yfinance


In [ ]:
# 2.3. Quandl
# The original code used Quandl API, but due to access issues and deprecation of WIKI/FB dataset,
# we are using Yahoo Finance instead for historical stock data.


import yfinance as yf

# Download Facebook (Meta) stock data
data = yf.download("META", start="2015-01-01", end="2025-01-01")

# Show info
data.info()


### FRED

In [ ]:
start = datetime(2010, 1, 1)

end = datetime(2013, 1, 27)

gdp = web.DataReader('GDP', 'fred', start, end)

gdp.info()

In [ ]:
inflation = web.DataReader(['CPIAUCSL', 'CPILFESL'], 'fred', start, end)
inflation.info()

### Fama/French

In [ ]:
# Load the Fama-French 3-Factor dataset starting from 2010
# This dataset contains market, SMB (size), and HML (value) factor returns
# from Kenneth French's data library

from pandas_datareader import data
ds = data.DataReader("F-F_Research_Data_Factors", "famafrench", start='2010-01-01')

# Display the keys (tables) in the dataset, such as monthly data and description
print(ds.keys())


In [ ]:
ds = web.DataReader('5_Industry_Portfolios', 'famafrench')
print(ds['DESCR'])

### World Bank

In [ ]:
from pandas_datareader import wb
gdp_variables = wb.search('gdp.*capita.*const')
gdp_variables.head()

In [ ]:
wb_data = wb.download(indicator='NY.GDP.PCAP.KD', 
                      country=['US', 'CA', 'MX'], 
                      start=1990, 
                      end=2019)
wb_data.head()

### OECD

In [ ]:
from pandas_datareader import data as web
import matplotlib.pyplot as plt

# Download GDP data from FRED
df = web.DataReader('GDP', 'fred', start='2010', end='2019')

# Plot
df.plot(title='US GDP (2010–2019)', figsize=(12, 5))
plt.grid()
plt.show()


### EuroStat

In [ ]:
from pandas_datareader import data as web
import matplotlib.pyplot as plt

# US Unemployment Rate from FRED
df = web.DataReader('UNRATE', 'fred', start='2010', end='2020')
df.plot(title='US Unemployment Rate (2010–2020)', figsize=(12, 5))
plt.grid()
plt.show()


In [ ]:
df.head()



### Stooq

Google finance stopped providing common index data download. The Stooq site had this data for download for a while but is currently broken, awaiting release of [fix](https://github.com/pydata/pandas-datareader/issues/594)

In [ ]:
index_url = 'https://stooq.com/t/'
ix = pd.read_html(index_url)
len(ix)

In [ ]:
sp500_stooq = web.DataReader('^SPX', 'stooq')
sp500_stooq.info()

In [ ]:
sp500_stooq.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the S&P 500 closing prices
sp500_stooq.Close.plot(figsize=(14, 4))

# Remove top and right spines for a cleaner look
sns.despine()

# Adjust subplot spacing
plt.tight_layout()


In [ ]:
sp500_stooq.Close.plot(figsize=(14,4))
sns.despine()
plt.tight_layout()

### NASDAQ Symbols

In [ ]:
# -----------------------------------------
# ✅ Replacement for failed Nasdaq FTP data fetch
# -----------------------------------------
# The original code used this URL to fetch NASDAQ symbols:
# url = "https://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
# However, this method failed with a timeout (WinError 10060) due to network restrictions,
# server issues, or because the FTP server is no longer reliable or accessible.

# ✅ Instead, we're now using the Wikipedia page listing all S&P 500 companies,
# which is more stable, up-to-date, and easy to parse using pandas.
# -----------------------------------------

import pandas as pd
import requests
from io import StringIO  # ✅ Used to fix the FutureWarning from read_html

# Step 1: Define the URL of the Wikipedia page listing S&P 500 companies
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Step 2: Send an HTTP GET request to fetch the page content
response = requests.get(sp_url)

# Step 3: Wrap HTML content with StringIO to avoid FutureWarning
html_content = StringIO(response.text)

# Step 4: Use pandas to extract all tables from the HTML content
tables = pd.read_html(html_content)

# Step 5: The first table on the page (index 0) contains the S&P 500 constituents
sp500_constituents = tables[0]

# Step 6: Display the first 5 rows to confirm the structure of the table
print(sp500_constituents.head())

# Step 7 (Optional): Extract just the stock symbols for further use (like downloading price data)
symbols = sp500_constituents['Symbol'].tolist()
print("\nTotal symbols extracted:", len(symbols))
print("Sample symbols:", symbols[:10])


### Tiingo

Requires [signing up](https://api.tiingo.com/) and storing API key in environment

In [ ]:
df = web.get_data_tiingo('GOOG', api_key=os.getenv('TIINGO_API_KEY'))

In [ ]:
df.info()